In [1]:
import math
import os
import gc
import re
import sys
import time
import json
import random
import datetime
import traceback
import pickle
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
from scipy.optimize import leastsq
from sklearn.linear_model import LinearRegression
from scipy import sparse
from xpinyin import Pinyin
import numpy as np

## 一.品牌匹配

In [5]:
open_category = pd.read_csv('../tmp/train/open_category.csv')
car_autohome_all = pd.read_csv('../tmp/train/car_autohome_all.csv')
car_autohome_all = car_autohome_all.drop(['body','energy'],axis=1)
car_autohome_all = car_autohome_all.loc[:, ['brand_name','brand_slug','model_name','model_slug','detail_name','detail_slug','online_year','price_bn']]
car_autohome_all = car_autohome_all.sort_values(by=['brand_name','model_name','online_year','price_bn'])

# 汽车之家品牌
car_autohome_brand = car_autohome_all.loc[:, ['brand_name','brand_slug']].drop_duplicates(['brand_slug']).rename(columns={'brand_slug':'car_autohome_brand_id'}).reset_index(drop=True)
print('汽车之家品牌数量:', len(car_autohome_brand))

liyang_brand = open_category.loc[(open_category['parent'].isnull()), ['id','name']].rename(columns={'name':'brand_name'}).reset_index(drop=True)
print('力洋品牌数量:', len(liyang_brand))
brand = liyang_brand.merge(car_autohome_brand, how='left', on=['brand_name'])
brand = brand.sort_values(by=['brand_name'])
car_autohome_brand = car_autohome_brand.sort_values(by=['brand_name'])
car_autohome_brand.to_csv('../tmp/train/match_car_autohome_brand.csv', index=False)
brand.to_csv('../tmp/train/final_brand.csv', index=False)
brand.info()
print('请人工匹配未匹配上品牌.')

汽车之家品牌数量: 196
力洋品牌数量: 156
<class 'pandas.core.frame.DataFrame'>
Int64Index: 156 entries, 114 to 111
Data columns (total 3 columns):
id                       156 non-null int64
brand_name               156 non-null object
car_autohome_brand_id    112 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 4.9+ KB
请人工匹配未匹配上品牌.


## 二.车系匹配

In [28]:
brand = pd.read_csv('../tmp/train/final_brand.csv')

open_category = pd.read_csv('../tmp/train/open_category.csv')
open_model_detail = pd.read_csv('../tmp/train/open_model_detail.csv')
open_category_brand = open_category.loc[(open_category['parent'].isnull()), ['id','slug']].rename(columns={'slug':'parent'}).reset_index(drop=True)
brand = open_category_brand.merge(brand,how='left',on=['id'])
model = open_category.loc[(open_category['parent'].notnull()), ['id','name','slug','parent']].rename(columns={'name':'model_name',
                         'slug':'model_slug','id':'model_id'}).reset_index(drop=True)
model = model.merge(brand, how='left', on=['parent']).rename(columns={'id':'brand_id'})
model = model.loc[:, ['brand_id','model_id','brand_name','model_name','model_slug','car_autohome_brand_id']]
print('力洋车系数量:', len(model))

# 汽车之家车系
car_autohome_model = car_autohome_all.loc[:, ['brand_slug','model_slug','model_name']].drop_duplicates(['model_slug']).rename(columns={'model_slug':'car_autohome_model_id','brand_slug':'car_autohome_brand_id'}).reset_index(drop=True)
print('汽车之家车系数量:', len(car_autohome_model))

model = model.merge(car_autohome_model, how='left', on=['car_autohome_brand_id','model_name'])
model.loc[(model['car_autohome_brand_id'].isnull()), 'car_autohome_brand_id'] = -1
model.loc[(model['car_autohome_brand_id'] == -1), 'car_autohome_model_id'] = -1
model = model.sort_values(by=['brand_name','model_name'])
model.to_csv('../tmp/train/final_model.csv', index=False)

temp = car_autohome_all.loc[:, ['model_slug','brand_name','model_name']].drop_duplicates(['model_slug'])                                                                                                                            
temp.to_csv('../tmp/train/match_car_autohome_model.csv', index=False)
model.info()
# model.head()
# model = model.merge(brand, how='left', on=['parent']).rename(columns={'parent':'brand_slug'})

力洋车系数量: 1765
汽车之家车系数量: 1826
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1765 entries, 890 to 539
Data columns (total 7 columns):
brand_id                 1765 non-null int64
model_id                 1765 non-null int64
brand_name               1765 non-null object
model_name               1765 non-null object
model_slug               1765 non-null object
car_autohome_brand_id    1765 non-null float64
car_autohome_model_id    608 non-null float64
dtypes: float64(2), int64(2), object(3)
memory usage: 110.3+ KB


In [47]:
def process_model_name(df):
    text = df['model_name']
    text = text.replace(df['brand_name'],'')
    text = text.replace(' ', '')
    text = text.lower()
    return text

model = pd.read_csv('../tmp/train/final_model.csv')
part1 = model.loc[(model['car_autohome_model_id'].notnull()), :].reset_index(drop=True)
part2 = model.loc[(model['car_autohome_model_id'].isnull()), :].reset_index(drop=True)
part2['model_name'] = part2.apply(process_model_name, axis=1)
part2 = part2.drop(['car_autohome_model_id'], axis=1)
car_autohome_model = car_autohome_all.loc[:, ['brand_slug','brand_name','model_slug','model_name']].drop_duplicates(['model_slug']).rename(columns={'model_slug':'car_autohome_model_id','brand_slug':'car_autohome_brand_id'}).reset_index(drop=True)
car_autohome_model['model_name'] = car_autohome_model.apply(process_model_name, axis=1)
car_autohome_model = car_autohome_model.drop(['brand_name'], axis=1)
part2 = part2.merge(car_autohome_model, how='left', on=['car_autohome_brand_id','model_name'])
final = part1.append(part2, sort=False)
final = final.sort_values(by=['brand_name','model_name'])
final.to_csv('../tmp/train/final_model.csv', index=False)
part2.head()

,brand_id,model_id,brand_name,model_name,model_slug,car_autohome_brand_id,car_autohome_model_id
0,1851,1881,Pagani,zonda,bseries_964,61,NaN
1,1346,1203,一汽,佳宝,jiabao,110,NaN
2,1346,1295,一汽,夏利2000,xiali2000,110,NaN
3,1346,1291,一汽,夏利a+,xiali-ajia,110,NaN
4,1346,1292,一汽,夏利n3,xiali-n3,110,NaN


## 三.款型匹配

In [17]:
open_model_detail = pd.read_csv('../tmp/train/open_model_detail.csv', low_memory=False).rename(columns={'year':'online_year','global_slug':'model_slug'})
# open_model_detail = open_model_detail.drop(['volume','control'],axis=1)
final_brand = pd.read_csv('../tmp/train/final_brand.csv')
final_model = pd.read_csv('../tmp/train/final_model.csv')
final_model = final_model.loc[(final_model['car_autohome_model_id'].notnull())&(final_model['car_autohome_model_id'] != -1), :].reset_index(drop=True)
gpj_match_model = open_model_detail.loc[(open_model_detail['model_slug'].isin(list(set(final_model.model_slug.values)))), :]
gpj_match_model = gpj_match_model.merge(final_model.loc[:,['model_slug','car_autohome_brand_id','car_autohome_model_id']],how='left',on=['model_slug'])
print('匹配上车系的力洋款型:', gpj_match_model.info())

car_autohome_all = pd.read_csv('../tmp/train/car_autohome_all.csv')
car_autohome_all = car_autohome_all.drop(['manufacturer','body','energy'], axis=1)
print('汽车之家车系总量:',len(list(set(car_autohome_all.model_slug.values))))

# 汽车之家补充的车系
supplement_model = car_autohome_all.loc[~(car_autohome_all['model_slug'].isin(list(set(final_model.car_autohome_model_id.values)))), :].reset_index(drop=True)
match_model = car_autohome_all.loc[(car_autohome_all['model_slug'].isin(list(set(final_model.car_autohome_model_id.values)))), :].reset_index(drop=True)
match_model = match_model.rename(columns={'model_slug':'car_autohome_model_id','detail_slug':'car_autohome_detail_id'}).loc[:, ['car_autohome_model_id','car_autohome_detail_id','detail_name','price_bn','online_year']]
print('汽车之家补充的车系量:', len(list(set(supplement_model.model_slug.values))))
print('汽车之家匹配的车系量:', len(list(set(match_model.car_autohome_model_id.values))))

final_detail = gpj_match_model.merge(match_model,how='left',on=['car_autohome_model_id','online_year','price_bn'])

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34902 entries, 0 to 34901
Data columns (total 27 columns):
id                       34902 non-null int64
source_id                20629 non-null float64
checker_runtime_id       17134 non-null float64
old_dmodel               24210 non-null object
detail_model             34902 non-null object
model_detail_slug        34902 non-null object
price_bn                 34902 non-null float64
cont_vprice              29623 non-null float64
url                      17134 non-null object
model_slug               34902 non-null object
domain                   34902 non-null object
status                   34902 non-null object
online_year              34902 non-null int64
has_param                34902 non-null object
volume                   34902 non-null float64
vv                       29389 non-null float64
listed_year              34902 non-null int64
delisted_year            18157 non-null float64
control                  34777 non-null o

In [18]:
def cal_match_rate(df):
    a = []
    a.extend(df['detail_model'].replace(' ',''))
    b = []
    b.extend(df['detail_name'].replace(' ',''))
    return float('%.3f' % (len(set(a).intersection(set(b)))/len(set(a))))
    
dup_details = final_detail.groupby(['id'])['model_detail_slug'].count().reset_index()
dup_details = dup_details.loc[(dup_details['model_detail_slug'] > 1), :].reset_index(drop=True)
dup_details = final_detail.loc[(final_detail['id'].isin(list(set(dup_details.id.values)))), :].reset_index(drop=True)
# dup_details.info()

part1 = final_detail.loc[~(final_detail['id'].isin(list(set(dup_details.id.values)))), :].reset_index(drop=True)

result = pd.DataFrame()
for liyang_id in list(set(dup_details.id.values)):
    temp = dup_details.loc[(dup_details['id'] == liyang_id), :].reset_index(drop=True)
    temp['match_rate'] = temp.apply(cal_match_rate, axis=1)
    result = result.append(temp.loc[(temp['match_rate'] == max(list(set(temp.match_rate.values)))), :].reset_index(drop=True), sort=False)
result.reset_index(inplace=True, drop=True)

dup_details = result.groupby(['id'])['model_detail_slug'].count().reset_index()
dup_details = dup_details.loc[(dup_details['model_detail_slug'] > 1), :].reset_index(drop=True)
dup_details = result.loc[(result['id'].isin(list(set(dup_details.id.values)))), :].reset_index(drop=True)

part2 = result.loc[~(result['id'].isin(list(set(dup_details.id.values)))), :].reset_index(drop=True)
match_details = part1.append(part2, sort=False).reset_index(drop=True)
dup_carhome_details = match_details.groupby(['car_autohome_detail_id'])['car_autohome_brand_id'].count().reset_index()
dup_carhome_details = dup_carhome_details.loc[(dup_carhome_details['car_autohome_brand_id'] == 1), :]
match_details = match_details.loc[(match_details['car_autohome_detail_id'].isin(list(set(dup_carhome_details.car_autohome_detail_id.values)))), :]
match_details = match_details.drop(['match_rate'], axis=1)
match_details = match_details.loc[(match_details['car_autohome_detail_id'].notnull()), :]

## 四.品牌车系款型组装

In [19]:
open_category = pd.read_csv('../tmp/train/open_category.csv', low_memory=False)
gpj_brand = open_category.loc[(open_category['parent'].isnull()), :].reset_index(drop=True)
gpj_model = open_category.loc[(open_category['parent'].notnull()), :].reset_index(drop=True)
gpj_detail = pd.read_csv('../tmp/train/open_model_detail.csv', low_memory=False)
car_autohome_all = pd.read_csv('../tmp/train/car_autohome_all.csv')

final_brand = pd.read_csv('../tmp/train/final_brand.csv')
final_model = pd.read_csv('../tmp/train/final_model.csv')

In [20]:
# 补充品牌
def process_brand(df, column_name):
    return 'car_autohome_brand_'+str(df[column_name])

def process_model(df, column_name):
    return 'car_autohome_model_'+str(df[column_name])

def process_detail(df, column_name):
    return 'car_autohome_detail_'+str(df[column_name])

def process_keywords(df):
    p = Pinyin()
    keywords = p.get_pinyin(df['name'], '')
    final_keywords = ','+keywords+','+df['name']+','+keywords
    first_letter = p.get_initial(df['name'][0])
    return pd.Series([final_keywords,first_letter,keywords])

liyang_brand = final_brand.loc[(final_brand['car_autohome_brand_id'].notnull()), :].reset_index(drop=True)
temp = car_autohome_all.loc[(car_autohome_all['brand_slug'].isin(list(set(liyang_brand.car_autohome_brand_id.values)))), :]
liyang_brand = liyang_brand.loc[(liyang_brand['car_autohome_brand_id'].isin(list(set(temp.brand_slug.values)))),:]
supplement_brand = list(set(car_autohome_all.brand_slug.values).difference(set(temp.brand_slug.values)))
supplement_brand = car_autohome_all.loc[(car_autohome_all['brand_slug'].isin(supplement_brand)), ['brand_slug','brand_name']]
supplement_brand = supplement_brand.drop_duplicates(['brand_slug','brand_name']).reset_index(drop=True)
supplement_brand = supplement_brand.rename(columns={'brand_name':'name','brand_slug':'slug'})
supplement_brand['car_autohome_brand_id'] = supplement_brand['slug']
supplement_brand['slug'] = supplement_brand.apply(process_brand,args=('slug',), axis=1)
supplement_brand[['keywords', 'first_letter', 'pinyin']] = supplement_brand.apply(process_keywords, axis=1)

gpj_brand = gpj_brand.loc[(gpj_brand['id'].isin(list(set(liyang_brand.id.values)))), :].reset_index(drop=True)
gpj_brand = gpj_brand.merge(final_brand.loc[:, ['id','car_autohome_brand_id']],how='left',on=['id'])
gpj_brand = gpj_brand.append(supplement_brand, sort=False).reset_index(drop=True)
gpj_brand['has_detailmodel'] = 0
gpj_brand['units'] = 0
gpj_brand['score'] = 0
gpj_brand['status'] = 'Y'
gpj_brand['popular'] = 'B'
gpj_brand['on_sale'] = '1'
gpj_brand.to_csv('../tmp/train/combine_brand.csv', index=False)
print('补充品牌数量:', len(supplement_brand))
print('力洋匹配上品牌数量:', len(liyang_brand))
print('汽车之家品牌数量:', len(list(set(car_autohome_all.brand_slug.values))))

补充品牌数量: 52
力洋匹配上品牌数量: 144
汽车之家品牌数量: 196


In [21]:
# 补充车系
def process_brand_step2(df, car_autohome_model, gpj_brand):
    car_autohome_brand_id = car_autohome_model.loc[(car_autohome_model['car_autohome_model_id'] == df['car_autohome_model_id']), 'car_autohome_brand_id'].values
    if len(car_autohome_brand_id) == 0:
        return 'car_autohome_brand_'+str(df['car_autohome_brand_id'])
    slug = gpj_brand.loc[(gpj_brand['car_autohome_brand_id'] == car_autohome_brand_id[0]), 'slug'].values[0]
    return slug

final_model = final_model.drop(['car_autohome_brand_id'],axis=1)
temp_copy = final_brand.loc[:, ['id','car_autohome_brand_id']].rename(columns={'id':'brand_id'})
final_model = final_model.merge(temp_copy,how='left',on=['brand_id'])
liyang_model = final_model.loc[(final_model['car_autohome_model_id'].notnull())&(final_model['car_autohome_model_id'] != -1), :].reset_index(drop=True)
gpj_model = gpj_model.loc[(gpj_model['id'].isin(list(set(liyang_model.model_id.values)))),:].reset_index(drop=True)
temp_copy = final_model.loc[:, ['model_slug','car_autohome_model_id']].rename(columns={'model_slug':'slug'})
gpj_model = gpj_model.merge(temp_copy, how='left', on=['slug'])

supplement_model_temp = supplement_model.loc[:, ['brand_slug','model_name','model_slug']]
supplement_model_temp = supplement_model_temp.drop_duplicates(['brand_slug','model_name','model_slug'])
supplement_model_temp = supplement_model_temp.rename(columns={'brand_slug':'parent', 'model_name':'name', 'model_slug':'slug'})
supplement_model_temp['car_autohome_model_id'] = supplement_model_temp['slug']
supplement_model_temp['slug'] = supplement_model_temp.apply(process_model,args=('slug',), axis=1)
supplement_model_temp[['keywords', 'first_letter', 'pinyin']] = supplement_model_temp.apply(process_keywords, axis=1)
supplement_model_temp = supplement_model_temp.drop(['first_letter'], axis=1)
merge = car_autohome_all.loc[:, ['model_slug', 'body', 'manufacturer']].drop_duplicates(['model_slug', 'body', 'manufacturer']).rename(columns={'model_slug':'car_autohome_model_id','body':'classified','manufacturer':'mum'}).reset_index(drop=True)
supplement_model_temp = supplement_model_temp.merge(merge, how='left', on=['car_autohome_model_id'])
supplement_model_temp['normalized_name'] = supplement_model_temp['name']

gpj_model = gpj_model.append(supplement_model_temp, sort=False).reset_index(drop=True)
car_autohome_model = car_autohome_all.loc[:,['brand_slug','model_slug']].drop_duplicates(['brand_slug','model_slug']).rename(columns={'model_slug':'car_autohome_model_id','brand_slug':'car_autohome_brand_id'}).reset_index(drop=True)
gpj_model = gpj_model.merge(car_autohome_model, how='left', on=['car_autohome_model_id'])
gpj_model['parent'] = gpj_model.apply(process_brand_step2,args=(car_autohome_model, gpj_brand), axis=1)
gpj_model = gpj_model.drop(['car_autohome_brand_id'], axis=1)
gpj_model['has_detailmodel'] = 0
gpj_model['units'] = 0
gpj_model['score'] = 0
gpj_model['status'] = 'Y'
gpj_model['popular'] = 'B'
gpj_model['on_sale'] = '1'
gpj_model.to_csv('../tmp/train/combine_model.csv', index=False)
print('补充车系数量:', len(list(set(supplement_model_temp.slug.values))))
print('力洋匹配上车系数量:', len(liyang_model))
print('汽车之家车系数量:', len(list(set(car_autohome_all.model_slug.values))))
temp_copy = gpj_brand.loc[(gpj_brand['id'].notnull()), ['id','slug']].reset_index(drop=True)
part1 = gpj_model.loc[(gpj_model['parent'].isin(list(temp_copy.slug.values))), :]

temp_copy = gpj_brand.loc[(gpj_brand['id'].isnull()), ['id','slug']].reset_index(drop=True)
part2 = gpj_model.loc[(gpj_model['parent'].isin(list(temp_copy.slug.values))), :]
print('公平价品牌:', len(list(set(part1.parent.values))), '纯粹新增品牌:', len(list(set(part2.parent.values))))
print('公平价品牌车系(包括新增车系):', len(part1), '纯粹新增品牌车系:', len(part2))

补充车系数量: 313
力洋匹配上车系数量: 1513
汽车之家车系数量: 1826
公平价品牌: 144 纯粹新增品牌: 52
公平价品牌车系(包括新增车系): 1627 纯粹新增品牌车系: 199


In [22]:
# 补充款型
def process_model_step2(df, data, column_name):
    temp = data.loc[(data['car_autohome_model_id'] == df['model_slug']), 'slug'].values
    if len(temp) == 0:
        return 'car_autohome_model_'+str(df[column_name])
    else:
        return temp[0]
    
liyang_detail = match_details.copy()
supplement_detail = car_autohome_all.loc[~(car_autohome_all['detail_slug'].isin(list(set(liyang_detail.car_autohome_detail_id.values)))), :]
supplement_detail = supplement_detail.loc[:, ['model_slug','detail_slug','detail_name','price_bn','online_year','volume','control','volume_extend','emission_standard','listed_year']]
supplement_detail = supplement_detail.rename(columns={'detail_slug':'car_autohome_detail_id',
                                                      'detail_name':'detail_model'})
supplement_detail['model_slug'] = supplement_detail.apply(process_model_step2, args=(gpj_model, 'model_slug',), axis=1)
supplement_detail['model_detail_slug'] = supplement_detail.apply(process_detail, args=('car_autohome_detail_id',), axis=1)

gpj_detail = liyang_detail.append(supplement_detail, sort=False)
gpj_detail = gpj_detail.rename(columns={'model_slug':'global_slug','online_year':'year','model_detail_slug':'detail_model_slug'})
gpj_detail = gpj_detail.drop(['car_autohome_brand_id','car_autohome_model_id','detail_name'],axis=1)
gpj_detail['created_on'] = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
gpj_detail['status'] = 'Y'
gpj_detail['has_param'] = 'Y'
gpj_detail.to_csv('../tmp/train/combine_detail.csv', index=False)
print('补充款型数量:', len(supplement_detail))
print('力洋匹配上款型数量:', len(liyang_detail))
print('汽车之家款型数量:', len(list(set(car_autohome_all.detail_slug.values))))

补充款型数量: 9628
力洋匹配上款型数量: 21809
汽车之家款型数量: 31437


## 五.拷贝款型库到api包

In [23]:
def process_mum(df):
    text = df['mum']
    regex = re.compile("\(进口\)")
    if len(regex.findall(text)) != 0:
        text = '进口' + re.sub("\(进口\)", '', text)
    text = text.replace('・','')
    text = text.replace('-','')
    text = text.replace('一汽大众奥迪','一汽奥迪')
    text = text.replace('北京汽车','北汽')
    text = text.replace('上汽通用','通用')
    text = text.replace('北汽昌河','昌河汽车')
    text = text.replace('上汽大众','上海大众')
    text = text.replace('福田汽车','福田')
    text = text.replace('一汽海马','海马汽车')
    text = text.replace('Jeep','吉普')
    text = text.replace('进口路虎','路虎')
    text = text.replace('力帆汽车','重庆力帆')
    text = text.replace('领克汽车','领克')
    text = text.replace('东风启辰','东风日产')
    text = text.replace('蔚来汽车','蔚来')
    text = text.replace('吉利全球鹰','吉利康迪')
    
    return text

evaluation_predict_path = '/home/ml/ProgramProject/evaluation-predict/tmp/train/'

gpj_brand = pd.read_csv('../tmp/train/combine_brand.csv')
gpj_model = pd.read_csv('../tmp/train/combine_model.csv')
gpj_detail = pd.read_csv('../tmp/train/combine_detail.csv')

gpj_model['mum'] = gpj_model.apply(process_mum, axis=1)
gpj_model.to_csv('../tmp/train/combine_model.csv', index=False)

gpj_brand.to_csv(evaluation_predict_path + 'combine_brand.csv', index=False)
gpj_model.to_csv(evaluation_predict_path + 'combine_model.csv', index=False)
gpj_detail.to_csv(evaluation_predict_path + 'combine_detail.csv', index=False)

## 六.新款型匹配

In [24]:
open_model_detail = pd.read_csv('../tmp/train/open_model_detail.csv', low_memory=False).rename(columns={'year':'online_year','global_slug':'model_slug'})
# open_model_detail = open_model_detail.drop(['volume','control'],axis=1)
final_brand = pd.read_csv('../tmp/train/final_brand.csv')
final_model = pd.read_csv('../tmp/train/final_model.csv')
final_model = final_model.loc[(final_model['car_autohome_model_id'].notnull())&(final_model['car_autohome_model_id'] != -1), :].reset_index(drop=True)
gpj_match_model = open_model_detail.loc[(open_model_detail['model_slug'].isin(list(set(final_model.model_slug.values)))), :]
gpj_match_model = gpj_match_model.merge(final_model.loc[:,['model_slug','car_autohome_brand_id','car_autohome_model_id']],how='left',on=['model_slug'])

add_new_detail = pd.read_csv('../tmp/train/add_new_detail.csv')
add_new_detail = add_new_detail.drop(['manufacturer','body','energy'], axis=1)
print('汽车之家新增款型总量:',len(list(set(add_new_detail.detail_slug.values))))
print('汽车之家新增款型的车系总量:',len(list(set(add_new_detail.model_slug.values))))

# 汽车之家补充的车系
supplement_model = add_new_detail.loc[~(add_new_detail['model_slug'].isin(list(set(final_model.car_autohome_model_id.values)))), :].reset_index(drop=True)
match_model = add_new_detail.loc[(add_new_detail['model_slug'].isin(list(set(final_model.car_autohome_model_id.values)))), :].reset_index(drop=True)
match_model = match_model.rename(columns={'model_slug':'car_autohome_model_id','detail_slug':'car_autohome_detail_id'}).loc[:, ['car_autohome_model_id','car_autohome_detail_id','detail_name','price_bn','online_year']]
print('汽车之家补充的车系量:', len(list(set(supplement_model.model_slug.values))))
print('汽车之家匹配的车系量:', len(list(set(match_model.car_autohome_model_id.values))))

final_detail = match_model.merge(gpj_match_model,how='left',on=['car_autohome_model_id','online_year','price_bn'])
print(len(final_detail))

汽车之家新增款型总量: 553
汽车之家新增款型的车系总量: 116
汽车之家补充的车系量: 38
汽车之家匹配的车系量: 78
431


In [25]:
def cal_match_rate(df):
    a = []
    a.extend(df['detail_model'].replace(' ',''))
    b = []
    b.extend(df['detail_name'].replace(' ',''))
    return float('%.3f' % (len(set(a).intersection(set(b)))/len(set(a))))
    
dup_details = final_detail.groupby(['id'])['model_detail_slug'].count().reset_index()
dup_details = dup_details.loc[(dup_details['model_detail_slug'] > 1), :].reset_index(drop=True)
dup_details = final_detail.loc[(final_detail['id'].isin(list(set(dup_details.id.values)))), :].reset_index(drop=True)
# dup_details.info()

part1 = final_detail.loc[~(final_detail['id'].isin(list(set(dup_details.id.values)))), :].reset_index(drop=True)

result = pd.DataFrame()
for liyang_id in list(set(dup_details.id.values)):
    temp = dup_details.loc[(dup_details['id'] == liyang_id), :].reset_index(drop=True)
    temp['match_rate'] = temp.apply(cal_match_rate, axis=1)
    result = result.append(temp.loc[(temp['match_rate'] == max(list(set(temp.match_rate.values)))), :].reset_index(drop=True), sort=False)
result.reset_index(inplace=True, drop=True)

dup_details = result.groupby(['id'])['model_detail_slug'].count().reset_index()
dup_details = dup_details.loc[(dup_details['model_detail_slug'] > 1), :].reset_index(drop=True)
dup_details = result.loc[(result['id'].isin(list(set(dup_details.id.values)))), :].reset_index(drop=True)

part2 = result.loc[~(result['id'].isin(list(set(dup_details.id.values)))), :].reset_index(drop=True)
match_details = part1.append(part2, sort=False).reset_index(drop=True)
dup_carhome_details = match_details.groupby(['car_autohome_detail_id'])['car_autohome_brand_id'].count().reset_index()
dup_carhome_details = dup_carhome_details.loc[(dup_carhome_details['car_autohome_brand_id'] == 1), :]
match_details = match_details.loc[(match_details['car_autohome_detail_id'].isin(list(set(dup_carhome_details.car_autohome_detail_id.values)))), :]
match_details = match_details.drop(['match_rate'], axis=1)
match_details = match_details.loc[(match_details['car_autohome_detail_id'].notnull()), :]

combine_detail = pd.read_csv('../tmp/train/combine_detail.csv')
match_details = match_details.loc[~(match_details['id'].isin(list(combine_detail.id.values))), :]

In [26]:
open_category = pd.read_csv('../tmp/train/open_category.csv', low_memory=False)
gpj_brand = open_category.loc[(open_category['parent'].isnull()), :].reset_index(drop=True)
gpj_model = open_category.loc[(open_category['parent'].notnull()), :].reset_index(drop=True)
gpj_detail = pd.read_csv('../tmp/train/open_model_detail.csv', low_memory=False)
car_autohome_all = pd.read_csv('../tmp/train/add_new_detail.csv')

combine_brand = pd.read_csv('../tmp/train/combine_brand.csv')
combine_model = pd.read_csv('../tmp/train/combine_model.csv')
combine_detail = pd.read_csv('../tmp/train/combine_detail.csv')

# 补充品牌
def process_brand(df, column_name):
    return 'car_autohome_brand_'+str(df[column_name])

def process_model(df, column_name):
    return 'car_autohome_model_'+str(df[column_name])

def process_detail(df, column_name):
    return 'car_autohome_detail_'+str(df[column_name])

def process_keywords(df):
    p = Pinyin()
    keywords = p.get_pinyin(df['name'], '')
    final_keywords = ','+keywords+','+df['name']+','+keywords
    first_letter = p.get_initial(df['name'][0])
    return pd.Series([final_keywords,first_letter,keywords])

supplement_brand = list(set(car_autohome_all.brand_slug.values).difference(set(combine_brand.car_autohome_brand_id.values)))
supplement_brand = car_autohome_all.loc[(car_autohome_all['brand_slug'].isin(supplement_brand)), ['brand_slug','brand_name']]
supplement_brand = supplement_brand.drop_duplicates(['brand_slug','brand_name']).reset_index(drop=True)
supplement_brand = supplement_brand.rename(columns={'brand_name':'name','brand_slug':'slug'})
supplement_brand['car_autohome_brand_id'] = supplement_brand['slug']
supplement_brand['slug'] = supplement_brand.apply(process_brand,args=('slug',), axis=1)
supplement_brand[['keywords', 'first_letter', 'pinyin']] = supplement_brand.apply(process_keywords, axis=1)

gpj_brand = pd.DataFrame([], columns=gpj_brand.columns)
gpj_brand = gpj_brand.append(supplement_brand, sort=False)
gpj_brand['has_detailmodel'] = 0
gpj_brand['units'] = 0
gpj_brand['score'] = 0
gpj_brand['status'] = 'Y'
gpj_brand['popular'] = 'B'
gpj_brand['on_sale'] = '1'
gpj_brand.to_csv('../tmp/train/add_combine_brand.csv', index=False)
print('补充品牌数量:', len(supplement_brand))
print('汽车之家品牌数量:', len(list(set(car_autohome_all.brand_slug.values))))

补充品牌数量: 4
汽车之家品牌数量: 67


In [27]:
# 补充车系
def process_brand_step2(df, car_autohome_model, gpj_brand):
    car_autohome_brand_id = car_autohome_model.loc[(car_autohome_model['car_autohome_model_id'] == df['car_autohome_model_id']), 'car_autohome_brand_id'].values
    if len(car_autohome_brand_id) == 0:
        return 'car_autohome_brand_'+str(df['car_autohome_brand_id'])
    slug = gpj_brand.loc[(gpj_brand['car_autohome_brand_id'] == car_autohome_brand_id[0]), 'slug'].values[0]
    return slug
    
gpj_brand = combine_brand.append(gpj_brand, sort=False)
supplement_model = car_autohome_all.loc[~(car_autohome_all['model_slug'].isin(list(set(combine_model.car_autohome_model_id.values)))), :]
supplement_model_temp = supplement_model.loc[:, ['brand_slug','model_name','model_slug']]
supplement_model_temp = supplement_model_temp.drop_duplicates(['brand_slug','model_name','model_slug'])
supplement_model_temp = supplement_model_temp.rename(columns={'brand_slug':'parent', 'model_name':'name', 'model_slug':'slug'})
supplement_model_temp['car_autohome_model_id'] = supplement_model_temp['slug']
supplement_model_temp['slug'] = supplement_model_temp.apply(process_model,args=('slug',), axis=1)
supplement_model_temp[['keywords', 'first_letter', 'pinyin']] = supplement_model_temp.apply(process_keywords, axis=1)
supplement_model_temp = supplement_model_temp.drop(['first_letter'], axis=1)
merge = car_autohome_all.loc[:, ['model_slug', 'body', 'manufacturer']].drop_duplicates(['model_slug', 'body', 'manufacturer']).rename(columns={'model_slug':'car_autohome_model_id','body':'classified','manufacturer':'mum'}).reset_index(drop=True)
supplement_model_temp = supplement_model_temp.merge(merge, how='left', on=['car_autohome_model_id'])
supplement_model_temp['normalized_name'] = supplement_model_temp['name']


gpj_model = pd.DataFrame([], columns=gpj_model.columns)
gpj_model = gpj_model.append(supplement_model_temp, sort=False)
car_autohome_model = car_autohome_all.loc[:,['brand_slug','model_slug']].drop_duplicates(['brand_slug','model_slug']).rename(columns={'model_slug':'car_autohome_model_id','brand_slug':'car_autohome_brand_id'}).reset_index(drop=True)
gpj_model = gpj_model.merge(car_autohome_model, how='left', on=['car_autohome_model_id'])
gpj_model['parent'] = gpj_model.apply(process_brand_step2,args=(car_autohome_model, gpj_brand), axis=1)
gpj_model = gpj_model.drop(['car_autohome_brand_id'], axis=1)
gpj_model['has_detailmodel'] = 0
gpj_model['units'] = 0
gpj_model['score'] = 0
gpj_model['status'] = 'Y'
gpj_model['popular'] = 'B'
gpj_model['on_sale'] = '1'
gpj_model.to_csv('../tmp/train/add_combine_model.csv', index=False)
print('补充车系数量:', len(list(set(supplement_model_temp.slug.values))))
print('汽车之家车系数量:', len(list(set(car_autohome_all.model_slug.values))))

补充车系数量: 25
汽车之家车系数量: 116


In [28]:
# 补充款型
def process_model_step2(df, data, column_name):
    temp = data.loc[(data['car_autohome_model_id'] == df['model_slug']), 'slug'].values
    if len(temp) == 0:
        return 'car_autohome_model_'+str(df[column_name])
    else:
        return temp[0]
    
liyang_detail = match_details.copy()
gpj_model = combine_model.append(gpj_model, sort=False)
supplement_detail = car_autohome_all.loc[~(car_autohome_all['detail_slug'].isin(list(set(liyang_detail.car_autohome_detail_id.values)))), :]
supplement_detail = supplement_detail.loc[:, ['model_slug','detail_slug','detail_name','price_bn','online_year','volume','control','volume_extend','emission_standard','listed_year']]
supplement_detail = supplement_detail.rename(columns={'detail_slug':'car_autohome_detail_id',
                                                      'detail_name':'detail_model'})
supplement_detail['model_slug'] = supplement_detail.apply(process_model_step2, args=(gpj_model, 'model_slug',), axis=1)
supplement_detail['model_detail_slug'] = supplement_detail.apply(process_detail, args=('car_autohome_detail_id',), axis=1)
gpj_detail = liyang_detail.append(supplement_detail, sort=False)
gpj_detail = gpj_detail.rename(columns={'model_slug':'global_slug','online_year':'year','model_detail_slug':'detail_model_slug'})
gpj_detail = gpj_detail.drop(['car_autohome_brand_id','car_autohome_model_id','detail_name'],axis=1)
gpj_detail['created_on'] = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
gpj_detail['status'] = 'Y'
gpj_detail['has_param'] = 'Y'
gpj_detail = gpj_detail.loc[:, combine_detail.columns]
gpj_detail.to_csv('../tmp/train/add_combine_detail.csv', index=False)
print('补充款型数量:', len(supplement_detail))
print('力洋匹配上款型数量:', len(liyang_detail))
print('汽车之家款型数量:', len(list(set(car_autohome_all.detail_slug.values))))

补充款型数量: 429
力洋匹配上款型数量: 124
汽车之家款型数量: 553


## 七.拷贝款型库到api包

In [30]:
def process_mum(df):
    text = df['mum']
    regex = re.compile("\(进口\)")
    if len(regex.findall(text)) != 0:
        text = '进口' + re.sub("\(进口\)", '', text)
    text = text.replace('・','')
    text = text.replace('-','')
    text = text.replace('一汽大众奥迪','一汽奥迪')
    text = text.replace('北京汽车','北汽')
    text = text.replace('上汽通用','通用')
    text = text.replace('北汽昌河','昌河汽车')
    text = text.replace('上汽大众','上海大众')
    text = text.replace('福田汽车','福田')
    text = text.replace('一汽海马','海马汽车')
    text = text.replace('Jeep','吉普')
    text = text.replace('进口路虎','路虎')
    text = text.replace('力帆汽车','重庆力帆')
    text = text.replace('领克汽车','领克')
    text = text.replace('东风启辰','东风日产')
    text = text.replace('蔚来汽车','蔚来')
    text = text.replace('吉利全球鹰','吉利康迪')
    
    return text

evaluation_predict_path = '/home/ml/ProgramProject/evaluation-predict/tmp/train/'

gpj_brand = pd.read_csv('../tmp/train/combine_brand.csv')
gpj_model = pd.read_csv('../tmp/train/combine_model.csv')
gpj_detail = pd.read_csv('../tmp/train/combine_detail.csv')
print('当前品牌数:', len(gpj_brand))
print('当前车系数:', len(gpj_model))
print('当前款型数:', len(gpj_detail))
add_combine_brand = pd.read_csv('../tmp/train/add_combine_brand.csv')
add_combine_model = pd.read_csv('../tmp/train/add_combine_model.csv')
add_combine_detail = pd.read_csv('../tmp/train/add_combine_detail.csv')
add_combine_model['mum'] = add_combine_model.apply(process_mum, axis=1)

gpj_brand = gpj_brand.append(add_combine_brand, sort=False).reset_index(drop=True)
gpj_model = gpj_model.append(add_combine_model, sort=False).reset_index(drop=True)
gpj_detail = gpj_detail.append(add_combine_detail, sort=False).reset_index(drop=True)

print('新增之后品牌数:', len(gpj_brand))
print('新增之后车系数:', len(gpj_model))
print('新增之后款型数:', len(gpj_detail))
gpj_brand.to_csv('../tmp/train/combine_brand.csv', index=False)
gpj_model.to_csv('../tmp/train/combine_model.csv', index=False)
gpj_detail.to_csv('../tmp/train/combine_detail.csv', index=False)

gpj_brand.to_csv(evaluation_predict_path + 'combine_brand.csv', index=False)
gpj_model.to_csv(evaluation_predict_path + 'combine_model.csv', index=False)
gpj_detail.to_csv(evaluation_predict_path + 'combine_detail.csv', index=False)

add_combine_brand.to_csv(evaluation_predict_path + 'add_combine_brand.csv', index=False)
add_combine_model.to_csv(evaluation_predict_path + 'add_combine_model.csv', index=False)
add_combine_detail.to_csv(evaluation_predict_path + 'add_combine_detail.csv', index=False)

当前品牌数: 196
当前车系数: 1826
当前款型数: 31437
新增之后品牌数: 200
新增之后车系数: 1851
新增之后款型数: 31990


In [31]:
brand = gpj_brand.loc[:, ['name','slug']].rename(columns={'name':'brand_name','slug':'parent'})
model = gpj_model.loc[:, ['name','slug','parent']].rename(columns={'name':'model_name','slug':'global_slug'})
model = model.merge(brand, how='left', on=['parent'])
final = gpj_detail.merge(model,how='left',on=['global_slug'])
final = final.sort_values(by=['parent','global_slug','year','price_bn'])
final.to_csv('../tmp/train/man.csv')
final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31990 entries, 31046 to 25899
Data columns (total 29 columns):
id                        21933 non-null float64
source_id                 11586 non-null float64
checker_runtime_id        10461 non-null float64
old_dmodel                13808 non-null object
detail_model              31990 non-null object
detail_model_slug         31990 non-null object
price_bn                  31990 non-null float64
cont_vprice               18014 non-null float64
url                       10461 non-null object
global_slug               31990 non-null object
domain                    21933 non-null object
status                    31990 non-null object
year                      31990 non-null int64
has_param                 31990 non-null object
volume                    31990 non-null float64
vv                        17994 non-null float64
listed_year               31990 non-null float64
delisted_year             8966 non-null float64
control         